In [2]:
# TODO: 
# - create Constants class?
# - brain vision recorder LSL
# - extend marker class?
# - introduce No-sound-'stimulus'
# - extend marker class + usage
# - 'change sequence' button
# - add error compability to Sequence + Execute Class
# - change errors to an info window which updates
# - on rename actualize right hand side

import random
import numpy as np

# not sure if this is useful
class Marker:
    
    counter = 0
    
    ################ standard definitions ######################
    def __init__(self,count=None,tag=None):
        if count != -1:
            if count == None: 
                value = Marker.counter
                Marker.counter += 1
            else: 
                value = count
                Marker.counter = max(Marker.counter,count+1)
        else:
            value = ''
        if tag != None:
            self.tag = tag + str(value)
        else:
            self.tag = str(value)

    def __str__(self):
        return 'Marker ' + self.tag
    
    def __repr__(self):
        return 'Marker ' + self.tag
    
    ################# CONTENT            #######################
        
    def identify(self): return self.tag

class Sequence:
    
        def __init__(self,probt,probd,length,prenov):
            
            
            self.probt = probt
            self.probd = probd
            self.length = length
            self.prenov = prenov
            
            Marker.counter = 0
            
            self.sequence = []
            self.expandedSeq = []
            self.amount_Target = 0
            self.amount_Novelty = {}
            self.amount_Standard = 0
        
            self.trails(probt,probd,length,prenov)
        
        def trails(self,Prob_T, Probs_D,Length,PreNov):

            amount_T = round(Length * Prob_T) # get amount of Targets from propability of a Target + amount of stimuli 
            amount_Ds = {} # fill dictonary with the amounts of every distractor stimuli same as above
            
            # this will be the sum of all probabilities but the standard one, so I can see if it 
            # is even possible to create a sequence
            probs_non_standard = Prob_T
            
            
            for name, prob in Probs_D.items():   
                amount = round(prob*Length)
                amount_Ds[name] = amount
                probs_non_standard += prob 
                
            self.probs = 1 - probs_non_standard

            if probs_non_standard * PreNov > 1: 
                print(self.errorMsg(probs_non_standard,PreNov))
                self.sequence = [] # TODO: raise exception here
                pass
            '''
            this happens if you for example say you want 2 standard stimuli previous to each novelty stimuli, the whole thing
            has to be 10 stimuli long and the two novelty stimuli have to appear with a probability of 0.4
            => length 10 => 8 novelty stimuli, each has to succeed 2 standard stimuli => 3 * 8 == 24 > 10!
            => that's simply not possible
            '''
            
            # save these values so we can call them outside of this function
            self.amount_Target = amount_T
            self.amount_Novelty = amount_Ds
            self.amount_Standard = Length - amount_T - sum(amount_Ds.values())

            sequence, amount_S = self.makeSeqs(PreNov,amount_T,amount_Ds,Length) 
            ''' 
            takes the amount of standard stimuli previous to novelty stimuli, the amount of target and distractor stimuli
            and the total amount of presented stimuli and returns a list of atomic sequences and the amount of standard stimuli
            which are still free to distribute
            '''

            self.sequence = self.permutations(amount_S * [['S']]  + sequence)
            '''
            we 'randomly' choose one permutation of the list of atomic sequences
            and return it
            '''


        def makeSeqs(self,pn,aT,aDs,length):

            seq_S = pn * ['S']

            seq_T = seq_S + ['T']
            seq_Ds = []

            for name, amount in aDs.items():
                seq_Ds += amount * [seq_S + [name]]

            result = seq_Ds + aT * [seq_T]
            return result, (length - (len(result) * (pn+1)))

        # returns one possible 'random' permutation of the given list
        def permutations(self,list):
            permus = random.sample(list,len(list))    
            return flatten(permus)

        # simply creates the error message when the given parameters in trails dont fit
        def errorMsg(self,probs,prenov):
            maxPreNov = round(1 / probs)
            maxProbs = float('%.2f'%(1 / prenov))
            return '\nIt is not possible to create a sequence of stimuli with the given probabilites for novelty stimuli and the given amount of buffer previous to novelty stimuli!\
                    \nEither change the buffer to {0} at max or the sum of novelty probabilites to {1} at max'.format(maxPreNov,maxProbs)
        
        
        # write oddball steps into a list, creating a sequence of actions which then can be 
        # easily executed
        # ISI = inter stimulus interval, PSI = pre stimulus interval, 
        # scaling_jitter = scaling factor for standard distribution, make it bigger!!!
        def expand(self,ISI,PSI,standard_dev,maxValue_jitter,minValue_jitter,mean_jitter):
            
            self.isi = ISI
            self.psi = PSI
            self.standard_dev = standard_dev
            self.maxValue_jitter = maxValue_jitter
            self.minValue_jitter = minValue_jitter
            self.mean_jitter = mean_jitter
            
            self.expandedSeq = self.expand_sequence(
                                               ISI,
                                               PSI,
                                               standard_dev,
                                               maxValue_jitter,
                                               minValue_jitter,
                                               mean_jitter)
            
        # maps te expand_single function over the whole sequence
        # since x consists of the pattern [[...],[...],...,[...]]
        # I have to flatten it into [.....]
        def expand_sequence(self,*args):
            self.count = 0
            x = map(lambda i: self.expand_single(i,*args),self.sequence)
            return flatten(list(x))

        # just concatinate the values 
        def expand_single(self,piece,ISI,PSI,sd,mav,miv,mean):
            realISI = ISI - PSI
            self.count += 1
            return ([Marker(count=self.count,tag = 'pre-' + piece)] +
                    [PSI]           + 
                    [Marker(count=self.count,tag = piece)] + 
                    [piece]         +  
                    [Marker(count=self.count,tag = 'post-' + piece)] + 
                    [realISI+self.calc_jitter(sd,mav,miv,mean)]
                   )

        # pulls a value from the gausian standard distribution, magnifies it with the 
        # scaling factor and then trunctuates it with the max and min values
        def calc_jitter(self,standard_dev, max_val, min_val, mean):
            return minmax(min_val,
                          max_val,
                          round(np.random.normal(mean,standard_dev)))
        
        
        def info_s(self):
            
            parameters = [self.length, self.prenov, self.isi, self.psi, self.standard_dev,
                          self.maxValue_jitter, self.minValue_jitter, self.mean_jitter, 
                          self.probt, self.probs, self.probd, self.sequence, self.expandedSeq
                         ]
            
            labels = ['Length', 'Pre-Nov-Stim', 'ISI', 'PSI', 'SD', 'MaxJitter', 'MinJitter',
                      'MeanJitter', 'ProbT', 'ProbS', 'ProbD', 'Sequence', 'ExpandedSequence'
                     ]
            
            self.info = ''
            
            for label, param in zip(labels,parameters):
                line = label + ' : ' + str(param) + '\n'
                self.info += line
            
            return self.info
        
        ## SETTERS ##
        
        def setSeq(self,seq):
            self.sequence = seq
        def setExpSeq(self,eseq):
            self.expandedSeq = eseq
        def setIsi(self,isi):
            self.isi = isi
        def setPsi(self,psi):
            self.psi = psi
        def setSd(self,sd):
            self.standard_dev = sd
        def setMaxJitter(self,maxjitter):
            self.maxValue_jitter = maxjitter
        def setMinJitter(self,minjitter):
            self.minValue_jitter = minjitter
        def setMeanJitter(self,meanjitter):
            self.mean_jitter = meanjitter
        def setProbS(self,standard):
            self.probs = standard
            
        ####### GETTERS ########
        
        def getTargetAmount(self):   return self.amount_Target
        def getStandardAmount(self): return self.amount_Standard
        def getNoveltyAmount(self):  return self.amount_Novelty        
        def getSeq(self):            return self.sequence
        def getExpandedSeq(self):    return self.expandedSeq
        def getIsi(self): return self.isi
        def getPsi(self): return self.psi
        def getSD(self): return self.standard_dev
        def getMaxV(self): return self.maxValue_jitter
        def getMinV(self): return self.minValue_jitter
        def getMean(self): return self.mean_jitter
        def getLength(self): return self.length
        def getPnss(self): return self.prenov
        
        
# concatinates a list of lists
def flatten(list):
    return [e for sublist in list for e in sublist]
    # how to read it:
    # for each sublist in the given list, for each element in the sublist, put that element into a list

# rounds a value up or down if it is not in between the given min value and max value
# minmax(5,10,6) = 6; minmax(5,10,11) =  10
def minmax(minv,maxv,x):
    t1 = min(x,maxv)
    t2 = max(t1,minv)
    return t2
        
############################################################################################################
############################################################################################################

# Testing

# define parameters
target_probability = 0.1
distractor_probabilites = {
    'd1' : 0.1,
    'none': 0.1
} 
sequence_length = 20
prenovelty_standard_stimuli = 1

# create sequence 
new_sequence = Sequence(target_probability,
                        distractor_probabilites,
                        sequence_length,
                        prenovelty_standard_stimuli)
print(new_sequence.getSeq())

print('\n\n')

standard_dev = 200
mean_jitter = 0
max_jitter = 200
min_jitter = -200
isi = 1500
psi = 500


new_sequence.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter)

print(new_sequence.getExpandedSeq())




#######################################################################################################
#######################################################################################################


['S', 'S', 'S', 'd1', 'S', 'none', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'S', 'none', 'S', 'S', 'd1', 'S', 'T']



[Marker pre-S1, 500, Marker S1, 'S', Marker post-S1, 852, Marker pre-S2, 500, Marker S2, 'S', Marker post-S2, 1121, Marker pre-S3, 500, Marker S3, 'S', Marker post-S3, 800, Marker pre-d14, 500, Marker d14, 'd1', Marker post-d14, 1200, Marker pre-S5, 500, Marker S5, 'S', Marker post-S5, 1086, Marker pre-none6, 500, Marker none6, 'none', Marker post-none6, 1200, Marker pre-S7, 500, Marker S7, 'S', Marker post-S7, 855, Marker pre-T8, 500, Marker T8, 'T', Marker post-T8, 867, Marker pre-S9, 500, Marker S9, 'S', Marker post-S9, 979, Marker pre-S10, 500, Marker S10, 'S', Marker post-S10, 860, Marker pre-S11, 500, Marker S11, 'S', Marker post-S11, 1016, Marker pre-S12, 500, Marker S12, 'S', Marker post-S12, 800, Marker pre-S13, 500, Marker S13, 'S', Marker post-S13, 833, Marker pre-S14, 500, Marker S14, 'S', Marker post-S14, 884, Marker pre-none15, 500, Marker none15, 'none', Marker

In [3]:
# Here I implemented the sequence execution
from pylsl import StreamOutlet, StreamInfo # you need to install this
from playsound import playsound # you need to install this library/ not a base library
from ast import literal_eval
import time
import os
import random
import datetime



# I inherit Sequence for structure
# inherit simply means that Execute has all the methods Sequence has plus some extra I define here
class Execute(Sequence):
    def __init__(self,soundDict,lsl_outlet,*args): # *args == probt,probd,length,prenov. I was just too lazy to write it out
        super().__init__(*args) # here I call the constructor for Sequence with probt, probd, lenght, prenov
        
        self.outlet = lsl_outlet
        self.sound = soundDict # here I save all the sounds to be used
        
    # executes the sequence of actions
    def execute(self):
        self.stopping = 0
        for action in self.getExpandedSeq():
            if self.stopping: break
            self.execute_action(action)

    def stop(self):
        self.stopping = 1
    
    # execute a single action
    def execute_action(self,action):
        if isinstance(action,Marker): # in case it is a Marker object do this            
            self.outlet.push_sample(['M%s' % action.identify()])
            print('Sent marker %s to brain visualizer via LSL' % action.identify())
        elif isinstance(action,str): # String object aka a sound file path
            sound = random.choice(self.sound[action])
            playsound(sound)
        elif isinstance(action,(int,float)): # int or float aka wait time
            time.sleep(action/1000) # time.sleep pauses the system for the number given in seconds, so I have to divide it by 1k
            
    def info_e(self):        
        info = self.info_s() + 'Sounds : ' + str(self.sound)
        return info
    
    def save(self,path):
        fullinfo = 'Time of creation :' + str(datetime.datetime.now()) + '\n' + self.info_e()
        handle = open(path,'w')
        handle.write(fullinfo)
        handle.close()
    
def fromString(string):
    lines = string.split('\n')
    length = pns = isi = psi = sd = maxjitter = minjitter = meanjitter = probt = probs = probd = seq = exseq = sounds = None 
    outlet =  StreamOutlet(StreamInfo('M1', 'Markers', 1, 0, 'string','12380128u182'))
    for line in lines:
        [iden, _, info] = line.split(' ',2)
        if iden == 'Length':
            length = int(info)
        elif iden == 'Pre-Nov-Stim':
            pns = int(info)
        elif iden == 'MaxJitter':
            maxjitter = float(info)
        elif iden == 'MinJitter':
            minjitter = float(info)
        elif iden == 'MeanJitter':
            meanjitter = float(info)
        elif iden == 'ProbS':
            probs = float(info)
        elif iden == 'SD':
            sd = float(info)
        elif iden == 'PSI':
            psi = float(info)
        elif iden == 'ISI':
            isi = float(info)
        elif iden == 'ProbT':
            probt = float(info)
        elif iden == 'ProbD':
            probd = literal_eval(info)
        elif iden == 'Sounds':
            sounds = literal_eval(info)
        elif iden == 'Sequence':
            seq = [i[1] for i in info.replace(" ","")[1:-1].split(',')]
        elif iden == 'ExpandedSequence':
            exseq = [identify(i.strip()) for i in info.strip()[1:-1].split(',')]

    new_seq = Execute(sounds,outlet,probt,probd,length,pns)
    new_seq.setSeq(seq)
    new_seq.setExpSeq(exseq)
    new_seq.setIsi(isi)
    new_seq.setPsi(psi)
    new_seq.setSd(sd)
    new_seq.setMaxJitter(maxjitter)
    new_seq.setMinJitter(minjitter)
    new_seq.setMeanJitter(meanjitter)
    new_seq.setProbS(probs)
    return new_seq
    
def identify(string):
    
    if string[:6] == 'Marker':
        [_,tag] = string.split(' ')
        return Marker(count=-1,tag=tag)
    elif string[0] == "'" and string[-1] == "'":
        return string[1:-1]        
    else:
        return float(string)
    
        
            
            
######################################################################################################### 
#######################   Testing
os.chdir('C:/Users/suuuj/Desktop/training room')
            
sounds = {
    'T' : ['baboon.mp3'],
    'D1' : ['bark.mp3'],
    'S' : ['batschatter.mp3']
}

probD = {
    'D1' : 0.1
}

standard_dev = 1
mean_jitter = 0
max_jitter = 200
min_jitter = -200
isi = 1500
psi = 500

outlet = StreamOutlet(StreamInfo('MyMarkerStream', 'Markers', 1, 0, 'string', 'myuidw43536'))

x = Execute(sounds,outlet,0.1,probD,20,1) # create Execute object

print(x.getSeq())

x.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter) # create actions sequence
x.save('test.txt')
inf = x.info_e()

print(x.getExpandedSeq())

seq = fromString(inf)

print(seq.getExpandedSeq())
#seq.execute()




['S', 'S', 'S', 'S', 'S', 'S', 'T', 'S', 'S', 'T', 'S', 'S', 'D1', 'S', 'S', 'S', 'S', 'D1', 'S', 'S']
[Marker pre-S1, 500, Marker S1, 'S', Marker post-S1, 1002, Marker pre-S2, 500, Marker S2, 'S', Marker post-S2, 999, Marker pre-S3, 500, Marker S3, 'S', Marker post-S3, 1001, Marker pre-S4, 500, Marker S4, 'S', Marker post-S4, 1001, Marker pre-S5, 500, Marker S5, 'S', Marker post-S5, 1002, Marker pre-S6, 500, Marker S6, 'S', Marker post-S6, 999, Marker pre-T7, 500, Marker T7, 'T', Marker post-T7, 1000, Marker pre-S8, 500, Marker S8, 'S', Marker post-S8, 1000, Marker pre-S9, 500, Marker S9, 'S', Marker post-S9, 1000, Marker pre-T10, 500, Marker T10, 'T', Marker post-T10, 1001, Marker pre-S11, 500, Marker S11, 'S', Marker post-S11, 998, Marker pre-S12, 500, Marker S12, 'S', Marker post-S12, 1000, Marker pre-D113, 500, Marker D113, 'D1', Marker post-D113, 1001, Marker pre-S14, 500, Marker S14, 'S', Marker post-S14, 998, Marker pre-S15, 500, Marker S15, 'S', Marker post-S15, 1000, Marker p

In [16]:
from tkinter import * # you need to install this / not in base library
from tkinter import filedialog # same here 
from pylsl import StreamInfo, StreamOutlet # same here
import Pmw # same here
import ntpath
import threading
import sys
   
# global Parameters
bwOuter = 5
bwInner = 3
epsilon =  0.0000000001
    
class GuiSequenceSingle(Frame):
    
    
    def __init__(self,parent):
        super().__init__(parent)
        self.parent = parent
    
    def display(self):
        
        self.selfframe = self.parent # I had to rewrite the code, and this is the shortest way
        
        self.pack(expand=YES,fill=BOTH)
        self.parent.minsize(1200,200)
        self.parent.title('Create Sequence')
        self.parent.option_add('*font',('verdana',8,'bold'))
        # self.master.protocol('WM_DELETE_WINDOW',self.quitBoth)
        
        self.frame = Frame(self.parent,borderwidth=bwOuter)
        self.frameStim = Frame(self.parent,borderwidth=bwOuter)
        self.frameStim.pack(fill=BOTH,expand=1)
        self.frame.pack(fill=BOTH,expand=1)
        
        self.initStimGui()
        self.initParamGui()
        self.initSeqEntry()
        if self.existing:
            self.existing = 0
            myDict = self.distDict
            myDict['T'] = self.target_prob
            myDict['S'] = self.leftover_prob
            for stim_type,sound in self.audios.items():
                sounds = ntpath.basename(sound[0])
                for s in sound[1:]: sounds = sounds + ',' + ntpath.basename(s)
                iden = 'Alias: %s ;Probabiliy: %.3f ;Audiofile: %s' % (self.convert(stim_type),myDict[stim_type],sounds)
                self.curStimLB.insert(END,iden)

    def quitBoth(self):
        self.selfframe.destroy()        
        sys.exit(0)
        
    def initParameters(self):
        
        self.seq = None
        
        self.leftover_prob = 1
        self.distCount = 0
        
        self.distDict = {}
        self.audios = {}
        
        self.target_prob = 0
        
        self.targetIden = 'Target'
        self.standardIden = 'Standard'
        self.distIden = 'Distractor'
        self.existing = 0
        
        # LSL
        
        self.outlet = None
        self.stream_id = '12380128u182'
        self.info = StreamInfo('M1', 'Markers', 1, 0, 'string',self.stream_id)
                            #  Name, Type, Channels, Frequency (0 means irregualr frequency), value type, stream id
        
        # Stimulus modifcation gui parameters
        
        self.curStimLB = None
        self.curStimLB_index = END
        
        # values for sequence calculation
        
        self.isi = None
        self.psi = None
        self.pnss = None
        self.jittersd = None
        self.jittermean = None
        self.jittermax = None
        self.jittermin = None
             
        # Param Labels + Entry Parameters
        
        self.initOddButton = None
        self.startButton = None
        self.pebal = None
        
            # entries 
            
        self.isi_entry        = None
        self.psi_entry        = None
        self.pnss_entry       = None
        self.jittersd_entry   = None
        self.jittermean_entry = None
        self.jittermax_entry  = None
        self.jittermin_entry  = None
        self.length           = None
        
        self.entries = None
        
            # labels + descriptions
            
        self.isi_label           = 'ISI'
        self.psi_label           = 'PSI'
        self.pnss_label          = 'Pre-Novelty-Stimulus Standard Stimuli'
        self.jittersd_label      = 'Jitter Standard Derivaiton'
        self.jittermean_label    = 'Jitter Mean Value'
        self.jittermax_label     = 'Jitter Max Value'
        self.jittermin_label     = 'Jitter Min Value'
        self.length_label        = 'Length'
        
        self.isi_desc        = 'Inter Stimulus Interval'
        self.psi_desc        = 'Pre Stimulus Interval'
        self.pnss_desc       = 'Amount of Standard Stimuli that need to preceed a Novelty Stimulus'
        self.jittersd_desc   = 'SD of gausian distribution used for creating the jitter'
        self.jittermean_desc = 'mean for the gausian distribution used for creating the jitter'
        self.jittermax_desc  = 'max value the jitter can have'
        self.jittermin_desc  = 'min value the jitter can have'
        self.length_desc     = 'Length of Oddball in auditory stimuli'
        
        # create Stimuli parameters
        
        self.csDialog = None
        
        self.bt_label_cs = 'Create Stimulus'
        self.bt_label_cancel = 'Cancel'
        
            # I want only one Target and one Standard Stimulus
        self.noTarget = True
        self.noStandard = True
        
        self.prob_entry = None
        self.prob_label = 'Probability'
        self.peval = 0
        
        self.openaudio_bt_label = 'Open Audio File'
        self.audio_entry = None
        
        self.filepath = ()
        
        # for update        
        self.stim_u = []
        
    def fromSeq(self,seq):
        self.seq = seq
        self.leftover_prob = seq.probs
        self.length = seq.length
        self.pnss_entry = seq.prenov
        self.isi_entry = seq.isi
        self.psi_entry = seq.psi
        self.jittersd_entry = seq.standard_dev
        self.jittermean_entry = seq
        self.jittermax_entry = seq.maxValue_jitter
        self.jittermin_entry = seq.minValue_jitter
        self.distCount = len(seq.sound) - 1
        self.distDict = seq.probd
        self.audios = seq.sound
        self.target_prob = seq.probt
        self.noTarget = self.noStandard = False
        self.existing = 1
            
    def convert(self,st): # with dict?
        if st == 'T': return 'Target'
        if st == 'S': return 'Standard'
        if st == 'Target': return 'T'
        if st == 'Standard': return 'S'
        if st[0] == 'd': return 'Distractor' + st[1:]
        if st[0:10] == 'Distractor': return 'd' + st[10:]
        
        
    def update(self):
        
        for stim in self.stim_u:
            self.curStimLB.insert(END,stim)
          
        if self.seq != None:            
            self.isi_entry.insert(END,self.seq.getIsi())
            self.psi_entry.insert(END,self.seq.getPsi())
            self.pnss_entry.insert(END,self.seq.getPnss())
            self.jittersd_entry.insert(END,self.seq.getSD())
            self.jittermean_entry.insert(END,self.seq.getMean())
            self.jittermax_entry.insert(END,self.seq.getMaxV())
            self.jittermin_entry.insert(END,self.seq.getMinV())
            self.length_entry.insert(END,self.seq.getLength())
        
    # Gui + functionality for creating and modifying stimuli
    
    def initStimGui(self):
        
        stim_frame = Frame(self.frameStim,borderwidth=bwOuter,relief=SUNKEN)
        stim_frame.pack(side=LEFT,expand=YES,fill=BOTH)
        
        # Buttons
        
        buttonBox = Pmw.ButtonBox(stim_frame,
                                  labelpos='nw',
                                  label_text='Modify Stimuli',
                                  frame_borderwidth=bwInner,
                                  frame_relief=GROOVE)
        buttonBox.pack(fill=X,padx=10,pady=10)
        
        buttonBox.add('Create Stimuli', command = self.createStim)
        buttonBox.add('Remove Stimuli', command = self.removeStim)
        buttonBox.add('Change Stimuli', command = self.changeStim)
        
        # Listbox for created Stimuli
        
        lbfx = Frame(stim_frame,borderwidth=bwInner)
        lbfx.pack(expand=YES,fill=BOTH)
        lbf = Frame(lbfx,
                    borderwidth = bwInner)
        lbf.pack(expand=YES,fill=BOTH)
        
        self.curStimLB = Listbox(lbf)
        self.curStimLB.pack(side=LEFT,expand=YES,fill=BOTH)
        
        scrolly = Scrollbar(lbf,command=self.curStimLB.yview)
        scrolly.pack(side=RIGHT,fill=BOTH)
        
        scrollx = Scrollbar(lbfx,orient='horizontal',command=self.curStimLB.xview)
        scrollx.pack(fill=X)
        
        self.curStimLB.configure(yscrollcommand = scrolly.set)
        
    def changeStim(self):
        k = 1
        if self.curStimLB.get(0,END) == ():
            errorWin(self.frame,'There is no stimulus to change')
            k = 0
        if self.curStimLB.curselection() == ():
            errorWin(self.frame, "No stimulus selected")
            k = 0
        if k: 
            self.curStimLB_index = self.curStimLB.curselection()            
            indf = self.curStimLB.get(self.curStimLB_index)

            prob, path = self.parse(indf.split(' ',5))

            self.peval = prob
            self.filepath = tuple(path.split(','))

            self.removeStim()
            self.createStim(opt=1)
            

    def parse(self,indf):
        if indf[1] == self.distIden:
            return float(indf[4]), indf[6]
        else:
            return float(indf[3]), indf[5]
    
    def removeStim(self):
        
        try:
            indx = self.curStimLB.curselection()
            stim_idens = self.curStimLB.get(indx).split()
            stim_type = stim_idens[1]
            self.curStimLB.delete(indx)
            
            if stim_type == self.standardIden:
                del(self.audios['S'])
                self.noStandard = True
            elif stim_type == self.targetIden:
                del(self.audios['T'])
                self.noTarget = True
                self.leftover_prob += self.target_prob
                self.target_prob = 0
                self.changeStandard()
            elif stim_type == self.distIden:
                iden = 'd%s' % stim_idens[2]
                self.leftover_prob += self.distDict[iden]
                del(self.distDict[iden])
                del(self.audios[iden])
                self.changeStandard()
        except TclError:
            errorWin(self.frame,'Select a stimulus you want to remove!')

    def createStim(self,opt=None):
        
        type_lb = None
        
        self.csDialog = Pmw.Dialog(self.frame,
                                  buttons=(self.bt_label_cs,self.bt_label_cancel),
                                  command = lambda i: self.execute(type_lb,i,opt=opt),
                                  title = 'Stimulus')
        
        # stim prob
        prob_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        prob_frame.pack(side=LEFT,fill=Y)
        Label(prob_frame,text='Maximum probability : %.3f' % self.leftover_prob).pack()
        self.prob_entry = Pmw.EntryField(prob_frame,
                                        labelpos=N,
                                        label_text=self.prob_label,
                                        validate = {
                                            'validator' : 'real',
                                            'max' : self.leftover_prob + epsilon
                                        })
        self.prob_entry.setvalue(self.peval)
        self.prob_entry.pack(side=LEFT,expand=1,fill=X,padx=5)
        
        # stim type
        stim_type_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        stim_type_frame.pack(side=LEFT,fill=Y)
        Label(stim_type_frame,text='Stimulus type',relief=FLAT,borderwidth=bwInner).pack(padx=3,pady=3)
        type_lb = Listbox(stim_type_frame,height=3)
        type_lb.insert(END,self.distIden)
        if self.noTarget: type_lb.insert(END,self.targetIden)
        if self.noStandard: type_lb.insert(END,self.standardIden)
        type_lb.pack(side=LEFT,expand=1,fill=X)
        
        # audio file
        audio_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        audio_frame.pack(side=LEFT,fill=BOTH,expand=1)
        self.audio_entry = Text(audio_frame,borderwidth=bwInner,height=2)
        files = ' '.join(self.filepath)
        self.audio_entry.insert(END,files)
        Button(audio_frame,text=self.openaudio_bt_label,command=self.filedialog).pack(padx=5,pady=5,expand=1,fill=X)
        self.audio_entry.pack(padx=5,pady=5,expand=1,fill=BOTH)
        # dialog.show() allows you still access to the parent, dialog.activate() does not
        self.csDialog.activate()
        
    def filedialog(self):
        files = filedialog.askopenfilenames()
        for file in files:
            self.audio_entry.insert(END,ntpath.basename(file) + ' ')
        self.filepath = self.filepath + files
        
    def execute(self,lb,iden,opt=None):
        if iden == self.bt_label_cs:
            k = 0
            try: 
                stimType = lb.get(lb.curselection())                
            except:
                errorWin(self.frame,'Specify a stimulus type!')  
                k = 1
            prob = float(self.prob_entry.getvalue())            
            if prob <= 0 and lb.get(lb.curselection()) != 'Standard':
                errorWin(self.frame,'Thou jest? Select a probability greater than 0!')
                k = 1
            if self.filepath == ():
                errorWin(self.frame,'Specify a file path!')
                k = 1
            if k == 0:
                pn,name = self.createEntry(stimType,prob)
                sounds = ','.join([ntpath.basename(path) for path in self.filepath])
                iden = 'Alias: %s ;Probabiliy: %.3f ;Audiofile: %s' % (name,pn,sounds)
                self.curStimLB.insert(self.curStimLB_index,iden)
                self.stim_u.append(iden)
                self.curStimLB_index = END
                self.peval = 0
                self.filepath = ()
                self.csDialog.destroy()
            
        elif iden == self.bt_label_cancel:
            if opt: pass
            self.peval = 0
            self.filepath = ()
            self.csDialog.destroy()
            
    def createEntry(self,t,p):
        fp = self.filepath
        
        if t == self.standardIden:
            self.audios['S'] = fp
            self.noStandard = False
            name = self.standardIden
            p = self.leftover_prob
        else:
            if t == self.targetIden:
                self.target_prob = p
                self.audios['T'] = fp
                self.noTarget = False
                name = self.targetIden
            elif t == self.distIden:
                iden = 'd%d' % self.distCount            
                name = self.distIden + ' %d' % self.distCount 
                self.distCount += 1
                self.distDict[iden] = p
                self.audios[iden] = fp   
                
            self.leftover_prob -= p
            self.changeStandard()
        
        return p, name
    
    def changeStandard(self):
        
        for i,label in enumerate(self.curStimLB.get(0,END)):
            i_lb = label.split()
            if i_lb[1] == self.standardIden:
                i_lb[3] = '%.3f' % self.leftover_prob
                new_label = ' '.join(i_lb)
                self.curStimLB.delete(i)
                self.curStimLB.insert(i,new_label)
                break
    
    # Gui + functionality for Parameter Input + Starting the actual Oddball
    
    def initParamGui(self):
        
        param_frame = Frame(self.frameStim,borderwidth=bwOuter,relief=SUNKEN)
        param_frame.pack(side=RIGHT,fill=BOTH,expand=YES)
        
        # Parameter Entries
        entry_frame = Frame(param_frame,relief=GROOVE,borderwidth=bwOuter)
        entry_frame.pack(expand = YES, fill=BOTH)
        
        self.pebal = Pmw.Balloon(entry_frame)
        
        self.isi_entry        = self.labelentry(entry_frame, self.isi_label,        self.isi_desc)
        self.psi_entry        = self.labelentry(entry_frame, self.psi_label,        self.psi_desc)
        self.pnss_entry       = self.labelentry(entry_frame, self.pnss_label,       self.pnss_desc)
        self.jittersd_entry   = self.labelentry(entry_frame, self.jittersd_label,   self.jittersd_desc)
        self.jittermean_entry = self.labelentry(entry_frame, self.jittermean_label, self.jittermean_desc)
        self.jittermax_entry  = self.labelentry(entry_frame, self.jittermax_label,  self.jittermax_desc)
        self.jittermin_entry  = self.labelentry(entry_frame, self.jittermin_label,  self.jittermin_desc)
        self.length_entry     = self.labelentry(entry_frame, self.length_label,     self.length_desc)
        
        self.entries = [
            self.isi_entry,
            self.psi_entry,
            self.pnss_entry,
            self.jittersd_entry,
            self.jittermean_entry,
            self.jittermax_entry,
            self.jittermin_entry,
            self.length_entry
        ]

        for entry in self.entries:
            entry.pack(fill=X, expand=YES, padx=10,pady=5)
            
        Pmw.alignlabels(self.entries)
        
        # Connect LSL + Start Exp Buttons
        but_frame = Frame(param_frame,borderwidth=bwInner,relief=GROOVE)
        but_frame.pack(expand=1,fill=X,padx=10,pady=10)
        
        Button(but_frame,text='Initialize LSL Outlet and Parameters',command=self.initLSL).pack(side=LEFT,expand=1,fill=X)
        self.initOddButton = Button(but_frame,text='Initialize Oddball Sequence',state=DISABLED,command=self.initSeq)
        self.initOddButton.pack(side=LEFT,expand=1,fill=X)
        self.startButton = Button(but_frame,text='Start Sequence',state=DISABLED,command = self.startSeq)
        self.startButton.pack(side=LEFT,expand=1,fill=X)
        self.saveButton = Button(but_frame,text='Save sequence and exit', state=DISABLED,command = self.selfframe.destroy)
        self.saveButton.pack(side=LEFT,expand=1,fill=X)
        
    def initLSL(self):
        k = 1
        
        try:
            self.isi = float(self.entries[0].getvalue())
            self.psi = float(self.entries[1].getvalue())
            self.pnss = int(self.entries[2].getvalue())
            self.jittersd = float(self.entries[3].getvalue())
            self.jittermean = float(self.entries[4].getvalue())
            self.jittermax = float(self.entries[5].getvalue())
            self.jittermin = float(self.entries[6].getvalue())
            self.length = int(self.entries[7].getvalue())
        except:
            errorWin(self.frame,'Some parameter fields were either not or falsely specified')
            k = 0
            
        # start LSL
        if k:
            self.outlet = StreamOutlet(self.info)
            self.initOddButton['state'] = NORMAL
            errorWin(self.frame,'Initialization Successful!')
            
    def initSeq(self):
        if not self.seq:        
            if (not self.noTarget) and (not self.noStandard):
                self.seq = Execute(self.audios,
                                   self.outlet,
                                   self.target_prob,
                                   self.distDict,
                                   self.length,
                                   self.pnss)
                self.seq.expand(self.isi,
                                self.psi,
                                self.jittersd,
                                self.jittermax,
                                self.jittermin,
                                self.jittermean
                               )
            else: 
                errorWin(self.frame,'You need to specify one target and standard stimulus in order to create a sequence!')
                pass
        self.seqEntry.setvalue(self.seq.getSeq())
        self.startButton['state'] = NORMAL
        self.saveButton['state'] = NORMAL
        errorWin(self.frame,'Sequence Creation successful!')
        
    def startSeq(self): 
        self.startButton['state'] = DISABLED
        self.initOddButton['state'] = DISABLED
        exe_thread = threading.Thread(target=self.seq.execute,args=(),daemon=False)
        exe_thread.start()        
        self.runningWin()
        errorWin(self.frame,'Sequence Execution Terminated! I (the program) reset all parameters because im an asshole :^)')
    
    def runningWin(self):
        self.running = Pmw.Dialog(self.frame,buttons=('Stop Sequence',),command = lambda _: self.stop())
        Label(self.running.interior(),text='Sequence is running ...').pack()
        self.running.activate()
        
    def stop(self):
        self.seq.stop()
        self.running.destroy()
    
    def initSeqEntry(self):
        seq_frame = Frame(self.frame,borderwidth=bwOuter,relief='sunken')
        seq_frame.pack(side=BOTTOM,anchor=S,fill=X,expand=1)
        Label(seq_frame,text='Current Sequence',borderwidth=bwInner,relief=GROOVE).pack(fill=X)
        self.seqEntry = Pmw.ScrolledText(seq_frame,text_height=10,text_state=DISABLED)
        self.seqEntry.pack(fill=BOTH,expand=1)
    
    # Getters
    def getSequence(self):
        return self.seq
    
    # Helper functions
    def labelentry(self,master,text='',description=''):
        w = Pmw.EntryField(master,labelpos = W,labelmargin=10,label_text=text)
        self.pebal.bind(w,description)
        return w

class GuiSequence(Frame):
    
    def __init__(self):
        Frame.__init__(self)
        self.pack(expand=YES,fill=BOTH)
        self.master.minsize(1200,300)
        self.master.title('Auditory Oddball')
        self.master.option_add('*font',('verdana',8,'bold'))
        
        self.initParameters()
        self.initCreateSeq()
        self.initSeqExec()
        
    def initParameters(self):
        
        self.activeSeq = {}
        self.allSeq = {}
        self.seqCounter = 0
    
    def initCreateSeq(self):
        
        cols = 2
        
        c_frame = Frame(self,borderwidth=bwOuter)
        c_frame.pack(expand=1,fill=BOTH)
        
        seq_frame = Frame(self,borderwidth=bwOuter,relief=SUNKEN)
        seq_frame.pack(expand=1,fill=BOTH)
        buttonBox = Pmw.ButtonBox(c_frame,
                                  labelpos='nw',
                                  label_text='Modify Sequences',
                                  frame_borderwidth=bwInner,
                                  frame_relief=GROOVE)
        buttonBox.pack(expand=1,fill=BOTH)
        
        buttonBox.add('Create Sequence', command = self.createSeq)
        buttonBox.add('Remove Sequence', command = self.removeSeq)
        buttonBox.add('Change Sequence', command = self.changeSeq)
        buttonBox.add('Rename Sequence', command = self.rename)
        
        # Listing
        
        label_lblist = Pmw.LabeledWidget(seq_frame,labelpos=N,label_text='All Sequences')
        label_lblist.pack(side=LEFT,expand=1,fill=BOTH)
        label_lblist.component('hull').configure(relief=SUNKEN,borderwidth=bwInner)
        
        self.lblist = Listbox(label_lblist.interior())
        self.lblist.pack(fill=BOTH,expand=1)
        
        # Shift Buttons
        shift_button_frame = Frame(seq_frame)
        shift_button_frame.pack(side=LEFT,expand=1,fill=BOTH)
        
        Button(shift_button_frame,text='Add to Right',command = self.addRight).pack(expand=1,fill=BOTH)
        #Button(shift_button_frame,text='Delete from Right', command = self.delRight).pack(expand=1,fill=BOTH)
        # Execute List
        
        exec_frame = Frame(seq_frame)
        exec_frame.pack(side=LEFT,expand=1,fill=BOTH)
        
        label_lbexec = Pmw.LabeledWidget(exec_frame,labelpos=N,label_text='Sequences to execute')
        label_lbexec.pack(expand=1,fill=BOTH)
        label_lbexec.component('hull').configure(relief=SUNKEN,borderwidth=bwInner)
        
        self.lbexec = Listbox(label_lbexec.interior())
        self.lbexec.pack(fill=BOTH,expand=1)
        
        Button(exec_frame,text='Execute Selected Sequences',command = self.execute).pack(side=BOTTOM,expand=1,fill=X)
    
    def initSeqExec(self):
        
        exec_frame = Frame(self,borderwidth=bwOuter,relief=RAISED)
        exec_frame.pack(fill=X)
        
        buttonBox = Pmw.ButtonBox(exec_frame,
                                 labelpos='nw',
                                 frame_borderwidth=bwInner,
                                 frame_relief=GROOVE)
        buttonBox.pack(fill=X,padx=10,pady=10)
        
        buttonBox.add('Save selected sequence',command = self.save)
        buttonBox.add('Load sequence',command = self.load)
        
    def createSeq(self): 
        new_win = Toplevel()
        
        cSeq = GuiSequenceSingle(new_win)
        cSeq.initParameters()
        cSeq.display()
        
        new_win.bind('<Destroy>',lambda _, seq=cSeq, win=new_win: self.saveSeq(seq,win))
        #new_win.protocol('WM_DELETE_WINDOW',lambda seq=cSeq, win=new_win: self.saveSeq(seq,win,0))
        
    def saveSeq(self,seqGui,window=None):
        if window:
            window.unbind('<Destroy>')
            window.destroy()
        if seqGui.getSequence() == None:
            errorWin(self,'No sequence was created!')
        else:
            name = 'Sequence%d' % self.seqCounter
            self.seqCounter += 1
            self.allSeq[name] = seqGui
            self.lblist.insert(END,name)
            errorWin(self,'A new sequence was created!')
        
    def removeSeq(self): 
        try: 
            indx = self.lblist.curselection()
            iden = self.lblist.get(indx)
            self.lblist.delete(indx)
            del(self.allSeq[iden])
        except TclError:
            try:
                indx = self.lbexec.curselection()
                iden = self.lbexec.get(indx)
                self.lbexec.delete(indx)
                try: del(self.activeSeq[iden])
                except KeyError: pass
            except TclError:
                errorWin(self,'Select a sequence to remove!')
                           
    def changeSeq(self):
            indx = self.lblist.curselection()
            iden = self.lblist.get(indx)
            
            new_win = Toplevel()
            gui = self.allSeq[iden]
            gui.__init__(new_win)
            gui.display()
            gui.update()
            
            new_win.bind('<Destroy>',lambda _, seq=gui, win=new_win: self.saveSeq(seq,win))
            
#     def delRight(self): 
#         try:
#             indx = self.lbexec.curselection()
#             iden = self.lbexec.get(indx)
#             self.lbexec.delete(indx)
#             del(self.activeSeq[iden])
#         except TclError:
#             errorWin(self,'Select a sequence to remove!')
            
    def addRight(self): 
        try:
            iden = self.lblist.get(self.lblist.curselection())
            self.lbexec.insert(END,iden)
            self.activeSeq[iden] = self.allSeq[iden]
        except TclError:
            errorWin(self,'Select a sequence to add!')
        
    def execute(self):         
        self.cont = 1
        for guiObj in self.activeSeq.values():
            if self.cont:
                seq = guiObj.getSequence()            
                cur_thread = threading.Thread(target= lambda : self.execute_h(seq),args=())
                cur_thread.start()
                self.runningWin(seq)
            
    def execute_h(self,seq):
        seq.execute()
        try: 
            self.running.destroy()
        except: pass
            
    def runningWin(self,seq):
        self.running = Pmw.Dialog(self,buttons=('Stop Sequence',),command = lambda _: self.stop(seq))
        Label(self.running.interior(),text='Sequence is running ...').pack()
        self.running.activate()
        
    def stop(self,seq):
        self.cont = 0
        seq.stop()
        self.running.destroy()
        
    def rename(self):
        try:
            indx = self.lblist.curselection()  
            ind = self.lblist.get(indx)
            self.renameWin = Pmw.Dialog(self,buttons=('Rename',),
                                       command = lambda _: self.rename_h(ind,indx))
            Label(self.renameWin.interior(), text='New Name').pack()
            self.rename_entry = Entry(self.renameWin.interior())
            self.rename_entry.pack(expand=1,fill=X)
        except TclError:
            errorWin(self,'Select a sequence from all sequences to rename!')
    
    def rename_h(self,ind,indx):
        new_name = self.rename_entry.get()
        if new_name == '':
            errorWin(self.renameWin.interior(),'Use a sensible name ;) ')
        else:  
            ind_info = self.allSeq[ind]
            del(self.allSeq[ind])
            self.allSeq[new_name] = ind_info
            self.lblist.delete(indx)
            self.lblist.insert(indx,new_name)
            self.renameWin.destroy()
            
    def save(self):
        try:
            indx = self.lblist.curselection()
            ind = self.lblist.get(indx)
            seq = self.allSeq[ind].getSequence()
            path = filedialog.asksaveasfilename()
            seq.save(path)
            errorWin(self,'Saved!')
        except TclError:
            errorWin(self,'Select a sequence from all sequences!')

    def load(self):
        path = filedialog.askopenfilename()
        if path != '':
            content = open(path,'r').read()
            seq = fromString(content)
            guiS = GuiSequenceSingle(self)
            guiS.initParameters()
            guiS.fromSeq(seq)
            self.saveSeq(guiS)

def errorWin(parent,msg):
    errorDia = Pmw.Dialog(parent,buttons=('Got it',),command=lambda _: errorDia.destroy())
    Label(errorDia.interior(),text=msg,relief=SUNKEN,borderwidth=bwInner,pady=5,padx=5).pack()
    errorDia.activate()
        
GuiSequence().mainloop()